In [2]:
import re
import numpy as np
import pandas as pd
import tabula
import warnings

warnings.filterwarnings('ignore')

pdf_file = "pdf3.pdf"

tabula.convert_into(pdf_file, output_path="output.json", output_format="json",
                    pages="all", stream=True)
tables = tabula.read_pdf(pdf_file, pages="all", output_format="json", stream=True, guess=True)

table_data = [i["data"] for i in tables]

# jsonFile_path = "output.json"
# df = pd.read_json(jsonFile_path)

extract_text_ls = []


def extract_text(data):
    for i in data:
        for j in i:
            for index, k in enumerate(j):
                text = k["text"]
                extract_text_ls.append(text)
    while "" in extract_text_ls:
        extract_text_ls.remove("")


extract_text(table_data)

index_to_slice = []

prog = re.compile(r"(?:\d{2}\.\s*){4}")
prog2 = re.compile(r"((\d{2}\.){2}-(\d{2}\.){2}-)")


def processing1(ls):
    for i, j in enumerate(ls):
        j = j.replace('  ', '-')
        j = j.replace(' ', '')
        ls[i] = j
        result1 = prog2.match(j)
        if result1 is not None:
            index_to_slice.append(i)
            continue
        elif "Blatt" in j:
            index_to_slice.append(i)
        elif "Konto" in j:
            index_to_slice.append(i)


processing1(extract_text_ls)

index_to_slice.sort()
subarrays = np.split(np.array(extract_text_ls), index_to_slice)

df = pd.DataFrame(subarrays)

new_cols = ['Date', 'Amount', 'Sender']

new_names_map = {df.columns[i]: new_cols[i] for i in range(len(new_cols))}
df.rename(new_names_map, axis=1, inplace=True)
df.dropna(subset=["Date"], inplace=True)
amount_ls = []
sender_ls = []
subject_ls = []


def indexed_row(df):
    for index, row in (df.iloc[:, [i for i in range(3, len(df.columns))]].iterrows()):
        row = row.str.cat(sep=" ")
        subject_ls.append(row)
    for index, row in (df.iloc[:, [0, 1, 2]].iterrows()):
        row = row.str.replace("-", " ")
        amount_ls.append(row[1])
        sender_ls.append(row[2])


indexed_row(df)

new_df = df.drop([i for i in range(3, len(df.columns))], axis=1)

date_ls = []
job_cat_ls = []


def append_ls(df):
    for j in df["Date"].values:
        result1 = prog2.match(j)
        if result1 is not None:
            date = result1.group()
            date_ls.append(date)
            str_list = j.replace(date, "")
            job_cat_ls.append(str_list)
        else:
            date_ls.append(j)
            job_cat_ls.append(j)


append_ls(new_df)


def final_df(df):
    df["Amount"] = amount_ls
    df["Sender"] = sender_ls
    df["Date"] =  [i.replace("-", " ").strip() for i in date_ls]
    df["Job"] = job_cat_ls
    df["Subject"] = [i.replace("-", " ") for i in subject_ls]

    return df


fin_df = final_df(new_df)

'''1. Date (first columns of PDF)
2. Job category (second column, first row)
3. Sender (second Column, second row)
4. Subject (second Column, third, fourth and fifth row, if exists)
5. the amount of money sent or received
6. Top Saldo (first row of page)
7. End Saldo (last row of page)'''


Got stderr: Jan 16, 2022 3:22:08 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:10 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:11 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:11 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:12 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:12 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:13 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:13 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:14 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:14 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased ensureDisplayProfile
Jan 16, 2022 3:22:

'1. Date (first columns of PDF)\n2. Job category (second column, first row)\n3. Sender (second Column, second row)\n4. Subject (second Column, third, fourth and fifth row, if exists)\n5. the amount of money sent or received\n6. Top Saldo (first row of page)\n7. End Saldo (last row of page)'

In [3]:
fin_df

,Date,Amount,Sender,Job,Subject
1,alter Kontostand vom 26.02.2021,"8.600,09 H",None,alter-Kontostand-vom-26.02.2021,
2,01.03. 01.03.,"31,49 S",Tele Columbus Kabel,Basislastschrift,"Service GmbH PYUR KD NR . 00008144002, RG NR..."
3,01.03. 01.03.,"87,59 S",SIGNAL IDUNA Gruppe,Basislastschrift,28636164 Krankenversicherung 00001 BIS 0321 ER...
4,01.03. 01.03.,"16,46 S",SIGNAL IDUNA Gruppe,Basislastschrift,28636164 Pflegepfl ichtversicherung 00001 BIS ...
5,01.03. 01.03.,"77,00 S",Stromio GmbH,Basislastschrift,ABSCHLAG Strom 03/21 VK: 123873859 gruenstrom ...
...,...,...,...,...,...
166,31.03. 31.03.,"450,00 H",Laura Witt,Dauerauftragsgutschr,Miete
167,31.03. 31.03.,"7,90 S","11,00000% Sollzinsen Limit 1 3103",Abschluss,"0,00H 11,00000% Sollzinsen Limit 2 3103 0,00H ..."
168,Kontoführungsentgelt 3103,"5,90S",VReG Vorteil Hausbankmodell 3103,Kontoführungsentgelt-3103,"0,00H Abschluss vom 01.01.2021 bis 31.03.2021"
169,für Konto 33391912,Rechnung Nr. 0033391912.20210331.002,USt.IdNr. DE 134777274,für-Konto-33391912,USt. befreite Finanzdienstleistung
